<a href="https://colab.research.google.com/github/Crudeoil0/3D-convolutional-neurol-network-cardiac-cell-segmentation/blob/main/3D_convolutional_neurol_network_cardiac_cell_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.15



In [ ]:
! pip install classification-models-3D
! pip install install efficientnet-3D
! pip install segmentation-models-3D

In [ ]:
! pip install patchify
! pip install torch
! pip install keras.utils

In [ ]:
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)

In [ ]:
import tensorflow as tf
device_name=tf.test.gpu_device_name()
if device_name !='/device:GPU:0':
  raise SystemError ('GPU device not found')
print ('Found GPU at :{}'.format(device_name))

In [ ]:
import segmentation_models_3D as sm 
from skimage import io 
from patchify import patchify, unpatchify 
import numpy as np
from matplotlib import pyplot as plt 
from keras import backend as K 
from keras.util import to_catalgical
from sklearn.model_selection import train_test_split

In [ ]:
image=io.imread ('')
img_patches=patchify(image, (64,64,64), step=64)

mask=imread ('')
mask_patches= patchify(mask,(64,64,64),step=64)

In [ ]:
print (img_patches.shape)

In [ ]:
plt.imshow(img_patches[1,2,3,:,:,32])
plt.imshow(mask_patches[1,2,3,:,:,32])

In [ ]:
input_img=np.reshape(img_patches, (-1, img_patches.shape[3], img_patches.shape[4], img_patches.shape[5]))
input_mask=np.reshape (mask_pathces, (-1, mask_patches.shape[3], mask_patches.shape[4], mask_pathces.shape[5])

print(input_img.shape) #n_patches, x, y, z

In [ ]:
n_classes=4 #catagories of color we observed on image

In [ ]:
#covert gery image to 3 channels by copying channel 3 times.
train_img=np.stack((input_img,)*3,axis=-1)
train_mask=np.expand_dims(input_mask,axis=4)

train_mask_cat = to_categorical (train_mask, num_classes=n_classes)

x_train, x_test, y_train, y_test = train_test_split(train_img, train_mask_cat, test_size= 0.30, random_xxx)




In [ ]:
# Loss Function and coefficients to be used during training;
def dice_coefficient (y_true, y_pred):
  smoothing_factor =1
  flat_y_true=K.flatten (y_true)
  flat_y_pred=K.flatten (y_pred)
  return (2.*K.sum(flat_y_true*flat_y_pred)+smoothing_factor)/(K.sum(flat_y_true)+K.sum(flat_y_pred))

def dice_coefficient_loss(y_true,y_pred):
  return 1 -dice_coefficient (y_true, y_pred)


In [ ]:
#Define parameters for our model.

encoder_weights='imagenet'
BACKBONE='vgg16'
activation='softmax'
patch_size=64
n_classes=4
channels=3

LR=0.0001
optim=keras.optimizers.Adam(LR)


dice_loss=sm.losses.DiceLoss(class_weights=np.array([0.25,0.25,0.25,0.25]))
focal_loss=sm.losses.CategoricalFocalLoss()
total_loss=dice_loss + (1*focal_loss)

metrics=[sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

In [ ]:
prepocess_input=sm.get_preprocessing(BACKBONE)

In [ ]:
X_train_prep=preprocess_input(X_train)
X_test_prep=preprocess_input(X_test)

In [ ]:
model=sm.Unet(BACKBONE, classes=n_classes,
              input_shape=(patch_size,patch_size,patch_size,channels),
              encoder_weights=encoder_weights,
              activation=activation)

model.compile(optimizer=optim, loss=total_loss, metrics=metrics)
print(model.summary())

In [ ]:
history=model.fit(X_train_prep,
                  y_train,
                  batch_size=8,
                  epochs=100,
                  verbose=1,
                  validation_data=(X_test_prep, y_test))

In [ ]:
model.save ('')

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
epochs=range(1, len(loss) +1)
plt.plot (epochs,loss,'y',lable='Training loss')
plt.plot (epochs,val_loss,'r',label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt,ylabel('Loss')
plt.legend()